### Create Embedding for the visualizer

This notebook shows how to create embeddings for the PCA / tSNE visualizer in tensorboard. 

In [49]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
import scipy
import cPickle
%matplotlib inline
import tensorflow.contrib.slim as slim
import tensorflow.contrib.slim.nets as nets
import tensorflow as tf
from tensorflow.contrib.tensorboard.plugins import projector
tf.__version__

'0.12.0-rc1'

### Loading of the data
The images can be obtained from: http://www.cs.toronto.edu/~kriz/cifar.html 

In [99]:
def unpickle(file):
    fo = open(file, 'rb')
    dict = cPickle.load(fo)
    fo.close()
    data = dict['data']
    imgs = np.transpose(np.reshape(data,(-1,32,32,3), order='F'),axes=(0,2,1,3)) #order batch,x,y,color
    y = np.asarray(dict['labels'], dtype='uint8')
    return y, imgs

In [100]:
y, imgs = unpickle('/Users/oli/Dropbox/data/CIFAR-10/cifar-10-batches-py/test_batch')
y.shape, imgs.shape

((10000,), (10000, 32, 32, 3))

### Defining the models
We use a training VGG16 network. The weights for the model can be downloaded as described in http://randomthoughtsonr.blogspot.de/2016/11/image-classification-in-r-using-trained.html

In [101]:
tf.reset_default_graph()
images = tf.placeholder(tf.float32, [None, None, None, 3])
imgs_scaled = tf.image.resize_images(images, (224,224))
slim.nets.vgg.vgg_16(imgs_scaled, is_training=False)
variables_to_restore = slim.get_variables_to_restore()
print('Number of variables to restore {}'.format(len(variables_to_restore)))
init_assign_op, init_feed_dict = slim.assign_from_checkpoint('/Users/oli/Dropbox/server_sync/tf_slim_models/vgg_16.ckpt', variables_to_restore)
sess = tf.Session()
sess.run(init_assign_op, init_feed_dict)

Number of variables to restore 32


In [83]:
g = tf.get_default_graph()
feed = g.get_tensor_by_name('Placeholder:0')
fetch = g.get_tensor_by_name('vgg_16/fc6/BiasAdd:0')

# Feeding 3 images through the net just for testing
feed_vals = imgs[0:3]
res = sess.run(fetch, feed_dict={feed:feed_vals})
np.shape(feed_vals), res.shape

((3, 32, 32, 3), (3, 1, 1, 4096))

### Getting the embedding
We now create a matrix with the embeddings

In [84]:
N = imgs.shape[0]
N = 3 #For testing
p = res.shape[3]

EMB = np.zeros((N, p), dtype='float32')
for i in range(N): #Of course you could do mini-batches
    EMB[i] = sess.run(fetch, feed_dict={feed: imgs[i:i+1,:]})

### Writing out the embedding matrix
We now write out the embedding matrix in the proper format. The 

In [95]:
# The embedding variable, which needs to be stored
# Note this must a Variable not a Tensor!
LOG_DIR = '/tmp/dumm'

embedding_var = tf.Variable(EMB,  name='Embedding_of_fc6')
sess.run(embedding_var.initializer)
summary_writer = tf.summary.FileWriter(LOG_DIR)
config = projector.ProjectorConfig()
embedding = config.embeddings.add()
embedding.tensor_name = embedding_var.name

# Comment out if you don't have metadata
embedding.metadata_path = os.path.join(LOG_DIR, 'metadata.tsv')

# Comment out if you don't want sprites
embedding.sprite.image_path = os.path.join(LOG_DIR, 'sprite.png')
embedding.sprite.single_image_dim.extend([imgs.shape[1], imgs.shape[1]])

projector.visualize_embeddings(summary_writer, config)
saver = tf.train.Saver([embedding_var])
saver.save(sess, os.path.join(LOG_DIR, 'model2.ckpt'), 1)

'/tmp/dumm/model2.ckpt-1'

### Creation of the meta data (labels)

In [96]:
names = ['plane','auto','bird','cat','deer','dog','frog','horse','ship','truck']
metadata_file = open(os.path.join(LOG_DIR, 'metadata.tsv'), 'w')
metadata_file.write('Name\tClass\n')
for i in range(N):
    metadata_file.write('%06d\t%s\n' % (i, names[y[i]]))
metadata_file.close()

### Creation of the images

In [97]:
# Taken from: https://github.com/tensorflow/tensorflow/issues/6322
def images_to_sprite(data):
    """Creates the sprite image along with any necessary padding

    Args:
      data: NxHxW[x3] tensor containing the images.

    Returns:
      data: Properly shaped HxWx3 image with any necessary padding.
    """
    if len(data.shape) == 3:
        data = np.tile(data[...,np.newaxis], (1,1,1,3))
    data = data.astype(np.float32)
    min = np.min(data.reshape((data.shape[0], -1)), axis=1)
    data = (data.transpose(1,2,3,0) - min).transpose(3,0,1,2)
    max = np.max(data.reshape((data.shape[0], -1)), axis=1)
    data = (data.transpose(1,2,3,0) / max).transpose(3,0,1,2)
    # Inverting the colors seems to look better for MNIST
    #data = 1 - data

    n = int(np.ceil(np.sqrt(data.shape[0])))
    padding = ((0, n ** 2 - data.shape[0]), (0, 0),
            (0, 0)) + ((0, 0),) * (data.ndim - 3)
    data = np.pad(data, padding, mode='constant',
            constant_values=0)
    # Tile the individual thumbnails into an image.
    data = data.reshape((n, n) + data.shape[1:]).transpose((0, 2, 1, 3)
            + tuple(range(4, data.ndim + 1)))
    data = data.reshape((n * data.shape[1], n * data.shape[3]) + data.shape[4:])
    data = (data * 255).astype(np.uint8)
    return data

In [98]:
sprite = images_to_sprite(imgs[0:3])
scipy.misc.imsave(os.path.join(LOG_DIR, 'sprite.png'), sprite)